## **SQL Server Trailing Spaces**

- Comparison of values: _The ANSI standard requires padding for the character strings used in comparisons so that their lengths match before comparing them. The padding directly affects the semantics of WHERE and HAVING clause predicates and other Transact-SQL string comparisons. For example, Transact-SQL considers the strings 'abc' and 'abc ' to be equivalent for most comparison operations._ - [INF: How SQL Server Compares Strings with Trailing Spaces (microsoft.com)](https://support.microsoft.com/en-us/topic/inf-how-sql-server-compares-strings-with-trailing-spaces-b62b1a2d-27d3-4260-216d-a605719003b0)
- Related to identifiers in SQL Server: _According to SQL Server, an identifier with trailing spaces is considered equivalent to the same identifier with those spaces removed. -_ [SQL Server Ignores Trailing Spaces In Identifiers | Michael J. Swart (michaeljswart.com)](https://michaeljswart.com/2014/09/sql-server-ignores-trailing-spaces-in-identifiers/)
- Keeping trailing spaces is controlled with SET ANSI\_PADDING: _If a value is being loaded into columns with a char, varchar, or varbinary data type, the padding or truncation of trailing blanks (spaces for char and varchar, zeros for varbinary) is determined by the SET ANSI\_PADDING setting defined for the column when the table was created. -_ [INSERT (Transact-SQL) - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/t-sql/statements/insert-transact-sql?view=sql-server-ver15#data-types)
- However, inserting data that is only differentiated by trailing spaces will fail when inserting into a column that enforces uniqueness: _An insertion into a column whose values must be unique will fail if you supply a value that is differentiated from existing values by trailing spaces only. The following strings will all be considered equivalent by a unique constraint, primary key, or unique index. Likewise, if you have an existing table with the data below and try to add a unique restriction, it will fail because the values are considered identical. -_ [Microsoft KB Archive/231830 - BetaArchive Wiki](https://www.betaarchive.com/wiki/index.php/Microsoft_KB_Archive/231830)
- Oracle versus SQL Server behavior: _In Oracle, values in primary key columns of type VARCHAR2 and NVARCHAR2 may differ only in trailing spaces. Oracle treats such values as not equal and allows you to use them as different primary keys. -_ [Trailing Spaces in PRIMARY KEY Columns - Oracle to SQL Server Migration - SQLines Tools](https://www.sqlines.com/oracle/trailing_spaces_pk_columns)

### **Setup a table**

In [28]:
DROP TABLE IF EXISTS [dbo].[testcustomertable]
GO

CREATE TABLE [dbo].[testcustomertable](
	[customerid] [bigint] NOT NULL,
	[customercode] [varchar](10) NOT NULL,
    [salespersoncode] [varchar](10) NOT NULL,
    [companyname] varchar(128) NOT NULL,
	[title] [varchar](8) NULL default('titlex'),
	[firstname] [nvarchar](64) NULL default('testfirstname'),
	[LastName] [nvarchar](64) NULL default('testlastname')
) 

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.074

### **Insert a row with a unique combination**

Insert #2 will contain _companyname_ with three trailing spaces.

In [6]:
-- #1
INSERT INTO [dbo].[testcustomertable](
    [customerid],
    [customercode],
    [salespersoncode],
    [companyname]
) VALUES (
    1,
    'custcode1',
    'salescode1',
    'companytest'
);

-- #2
INSERT INTO [dbo].[testcustomertable](
    [customerid],
    [customercode],
    [salespersoncode],
    [companyname]
) VALUES (
    1,
    'custcode1',
    'salescode1',
    'companytest   '
);

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.045

### **Review data**

LEN versus DATALENGTH - [LEN (Transact-SQL) - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/t-sql/functions/len-transact-sql?view=sql-server-ver15#remarks) - _LEN excludes trailing spaces. If that is a problem, consider using the DATALENGTH (Transact-SQL) function which does not trim the string. If processing a unicode string, DATALENGTH will return a number that may not be equal to the number of characters._

In [7]:
SELECT 
    [customerid],
    [customercode],
    [salespersoncode],
    [companyname],
    LEN([companyname]) AS [companyname_len],
    DATALENGTH([companyname]) AS [companyname_length] 
FROM [dbo].[testcustomertable];

(2 rows affected)

Total execution time: 00:00:00.080

customerid,customercode,salespersoncode,companyname,companyname_len,companyname_length
1,custcode1,salescode1,companytest,11,11
1,custcode1,salescode1,companytest,11,14


### **Compare**

[INF: How SQL Server Compares Strings with Trailing Spaces](https://support.microsoft.com/nl-nl/topic/inf-how-sql-server-compares-strings-with-trailing-spaces-b62b1a2d-27d3-4260-216d-a605719003b0): _SQL Server follows the ANSI/ISO SQL-92 specification (Section 8.2, \<Comparison Predicate\>, General rules #3) on how to compare strings with spaces. The ANSI standard requires padding for the character strings used in comparisons so that their lengths match before comparing them. The padding directly affects the semantics of WHERE and HAVING clause predicates and other Transact-SQL string comparisons. For example, Transact-SQL considers the strings 'abc' and 'abc ' to be equivalent for most comparison operations._

Below _select_ will return both rows from our example data. So, including the _companyname_ with trailing spaces.

In [8]:
SELECT *, DATALENGTH([companyname]) AS [companyname_datalength]
FROM [dbo].[testcustomertable]
WHERE [companyname] = 'companytest';

(2 rows affected)

Total execution time: 00:00:00.090

customerid,customercode,salespersoncode,companyname,title,firstname,LastName,companyname_datalength
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,11
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,14


Using a combination of search value and DATALENGTH can be used to get the unique value. 

However, be **aware** of the following: _If processing a unicode string, DATALENGTH will return a number that may not be equal to the number of characters._

In [9]:
-- Use datalength to limit the where-clause to obtain a specific value
SELECT *, DATALENGTH(companyname) AS companyname_datalength
FROM [dbo].[testcustomertable]
WHERE [companyname] = 'companytest   '
AND DATALENGTH([companyname]) = DATALENGTH('companytest   ');

(1 row affected)

Total execution time: 00:00:00.045

customerid,customercode,salespersoncode,companyname,title,firstname,LastName,companyname_datalength
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,14


This solution will be compatible with unicode, however be aware of column length.

In [10]:
-- Add a trailing character in the comparison
SELECT *, DATALENGTH(companyname) AS companyname_datalength
FROM [dbo].[testcustomertable]
WHERE [companyname] + 'x' = 'companytest   ' + 'x';

(1 row affected)

Total execution time: 00:00:00.084

customerid,customercode,salespersoncode,companyname,title,firstname,LastName,companyname_datalength
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,14


In [11]:
SELECT 
    [companyname],
    DATALENGTH([companyname]) AS [companyname_length],
    CASE WHEN [companyname] = 'companytest' THEN 'MATCH' ELSE 'NO MATCH' END [eq],
    CASE WHEN [companyname] + 'x' = 'companytestx' THEN 'MATCH' ELSE 'NO MATCH' END [eq_add_trailing_char],
    CASE WHEN [companyname] LIKE 'companytest' THEN 'MATCH' ELSE 'NO MATCH' END [like_without_spaces],
    CASE WHEN [companyname] LIKE 'companytest   ' THEN 'MATCH' ELSE 'NO MATCH' END AS [like_with_spaces]
FROM [dbo].[testcustomertable];

(2 rows affected)

Total execution time: 00:00:00.047

companyname,companyname_length,eq,eq_add_trailing_char,like_without_spaces,like_with_spaces
companytest,11,MATCH,MATCH,MATCH,NO MATCH
companytest,14,MATCH,NO MATCH,MATCH,MATCH


### **Create Unique index on <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">customerid, c</span><span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">ustomercode, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">salespersoncode, </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">companyname</span>**

[INF: Insertion of Strings Containing Trailing Spaces:](https://www.betaarchive.com/wiki/index.php/Microsoft_KB_Archive/231830) <span style="color: rgb(33, 37, 41); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Roboto, &quot;Helvetica Neue&quot;, Arial, &quot;Noto Sans&quot;, &quot;Liberation Sans&quot;, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; background-color: rgb(255, 255, 255);"><i>An insertion into a column whose values must be unique will fail if you supply a value that is differentiated from existing values by trailing spaces only. The following strings will all be considered equivalent by a unique constraint, primary key, or unique index. Likewise, if you have an existing table with the data below and try to add a unique restriction, it will fail because the values are considered identical.</i></span>

In [13]:
-- Will fail!
CREATE UNIQUE NONCLUSTERED INDEX [IDX_UNIQUECOMBINATION]
    ON [dbo].[testcustomertable]([customerid] ASC, [customercode] ASC, [salespersoncode] ASC, [companyname] ASC);


: Msg 1505, Level 16, State 1, Line 2
The CREATE UNIQUE INDEX statement terminated because a duplicate key was found for the object name 'dbo.testcustomertable' and the index name 'IDX_UNIQUECOMBINATION'. The duplicate key value is (1, custcode1, salescode1, companytest   ).

The statement has been terminated.

Total execution time: 00:00:00.064

### **Alternative 1 - Add computed column with trailing character**

Solution is compatible with unicode (datatype _nvarchar_, inserting _N'value'_)

In [17]:
-- Alternative one - Add trailing character
ALTER TABLE [dbo].[testcustomertable]
ADD [companyname_trailing_character] AS CAST([companyname] + 'x' AS VARCHAR(129));

Commands completed successfully.

Total execution time: 00:00:00.039

In [18]:
CREATE UNIQUE NONCLUSTERED INDEX [IDX_UNIQUECOMBINATION_1]
    ON [dbo].[testcustomertable]([customerid] ASC, [customercode] ASC, [salespersoncode] ASC, [companyname_trailing_character] ASC);

Commands completed successfully.

Total execution time: 00:00:00.036

In [19]:
-- Will fail!
-- #1
INSERT INTO [dbo].[testcustomertable](
    [customerid],
    [customercode],
    [salespersoncode],
    [companyname]
) VALUES (
    1,
    'custcode1',
    'salescode1',
    'companytest'
);

-- #2
INSERT INTO [dbo].[testcustomertable](
    [customerid],
    [customercode],
    [salespersoncode],
    [companyname]
) VALUES (
    1,
    'custcode1',
    'salescode1',
    'companytest   '
);

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.041

### **Alternative 2 - Add computed column with datalength of companyname**

Add the _companyname\_length_ column as computed column. Important to keep the following in mind: _If processing a unicode string, DATALENGTH will return a number that may not be equal to the number of characters._

The computed column can also be [persisted](https://docs.microsoft.com/en-us/sql/relational-databases/tables/specify-computed-columns-in-a-table?view=sql-server-ver15). This might have a [performance benefit](https://www.red-gate.com/simple-talk/databases/sql-server/performance-sql-server/computed-column-performance-in-sql-server/) over more space needed.

In [29]:
-- Alternative one - Add DATALENGTH as computed column
ALTER TABLE [dbo].[testcustomertable]
--ADD [companyname_length] AS DATALENGTH(companyname);
ADD [companyname_length] AS DATALENGTH(companyname) PERSISTED;

Commands completed successfully.

Total execution time: 00:00:00.032

Add the unique index including the _companyname\_length_ computed column.

In [30]:
-- Computed column [companyname_length] as part of the unique index
-- https://docs.microsoft.com/en-us/sql/relational-databases/indexes/indexes-on-computed-columns?view=sql-server-ver15
DROP INDEX IF EXISTS [IDX_UNIQUECOMBINATION_2] ON [dbo].[testcustomertable]

CREATE UNIQUE NONCLUSTERED INDEX [IDX_UNIQUECOMBINATION_2]
    ON [dbo].[testcustomertable]([customerid] ASC, [customercode] ASC, [salespersoncode] ASC, [companyname] ASC, [companyname_length] ASC);

Commands completed successfully.

Total execution time: 00:00:00.041

In [33]:
SELECT *
FROM [dbo].[testcustomertable];

(2 rows affected)

Total execution time: 00:00:00.043

customerid,customercode,salespersoncode,companyname,title,firstname,LastName,companyname_length
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,11
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,14


Insert existing data will **fail** with a duplicate key error.

In [34]:
-- #1
INSERT INTO [dbo].[testcustomertable](
    customerid,
    customercode,
    salespersoncode,
    companyname
) VALUES (
1,
'custcode1',
'salescode1',
'companytest'
);

-- #2
INSERT INTO [dbo].[testcustomertable](
    customerid,
    customercode,
    salespersoncode,
    companyname
) VALUES (
    1,
    'custcode1',
    'salescode1',
    'companytest   '
);

: Msg 2601, Level 14, State 1, Line 2
Cannot insert duplicate key row in object 'dbo.testcustomertable' with unique index 'IDX_UNIQUECOMBINATION_2'. The duplicate key value is (1, custcode1, salescode1, companytest, 11).

The statement has been terminated.

: Msg 2601, Level 14, State 1, Line 15
Cannot insert duplicate key row in object 'dbo.testcustomertable' with unique index 'IDX_UNIQUECOMBINATION_2'. The duplicate key value is (1, custcode1, salescode1, companytest   , 14).

The statement has been terminated.

Total execution time: 00:00:00.039

Adding a new set of unique records containing spaces works as expected.

In [35]:
-- #3
INSERT INTO [dbo].[testcustomertable](
    customerid,
    customercode,
    salespersoncode,
    companyname
) VALUES (
1,
'custcode2',
'salescode2',
'companytest'
);

-- #4
INSERT INTO [dbo].[testcustomertable](
    customerid,
    customercode,
    salespersoncode,
    companyname
) VALUES (
    1,
    'custcode2',
    'salescode2',
    'companytest   '
);

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.035

In [36]:
SELECT * 
FROM [dbo].[testcustomertable];

(4 rows affected)

Total execution time: 00:00:00.040

customerid,customercode,salespersoncode,companyname,title,firstname,LastName,companyname_length
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,11
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,14
1,custcode2,salescode2,companytest,titlex,testfirstname,testlastname,11
1,custcode2,salescode2,companytest,titlex,testfirstname,testlastname,14


Modify a record to identify that the computed column is updated.

In [37]:
-- Update test
UPDATE [dbo].[testcustomertable]
    SET [companyname] = 'testcompanytest'
WHERE [customercode] = 'custcode1' 
AND [salespersoncode] = 'salescode1'
AND [companyname] = 'companytest'
AND [companyname_length] = 11

(1 row affected)

Total execution time: 00:00:00.066

Review the combination being updated with _companyname_ to <span style="color: rgb(163, 21, 21); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">testcompanytest </span> and companyname\_length has been changed to 15 for this record.

In [38]:
SELECT * 
FROM [dbo].[testcustomertable]

(4 rows affected)

Total execution time: 00:00:00.047

customerid,customercode,salespersoncode,companyname,title,firstname,LastName,companyname_length
1,custcode1,salescode1,testcompanytest,titlex,testfirstname,testlastname,15
1,custcode1,salescode1,companytest,titlex,testfirstname,testlastname,14
1,custcode2,salescode2,companytest,titlex,testfirstname,testlastname,11
1,custcode2,salescode2,companytest,titlex,testfirstname,testlastname,14
